# util

> utilities for use in the challenges

In [ ]:
#| default_exp util

In [ ]:
#| export
# util functions go here
import numpy as np
import gdsfactory.simulation.gtidy3d as gt
import matplotlib.pyplot as plt
import gdsfactory as gf
from tqdm.notebook import tqdm
import meow as mw

gf.config.rich_output()
PDK = gf.generic_tech.get_generic_pdk()
PDK.activate()

nm = 1e-3

In [ ]:
#|export
def TE_pol_fraction(self):
        """TE polarization fraction according to Lumericals definition. (assuming a regular mesh)"""
        Ex_sq = np.abs(self._data["Ex"])**2
        Ey_sq = np.abs(self._data["Ey"])**2
        
        return np.sum(Ex_sq, axis=(0,1))/np.sum(Ex_sq+Ey_sq, axis=(0,1))


In [ ]:
#|export
#|hide
setattr(gt.modes.Waveguide, 'TE_pol_fraction', property(TE_pol_fraction))

## Caching
let's define some classes, that will allow us to efficiently cache the modes and interface s matrices

In [ ]:
#| exporti
import meow as mw
from functools import lru_cache
import json
from hashlib import md5
from meow import BaseModel, CrossSection, Mode
from pydantic import Field

In [ ]:
#| exporti
def dict_to_hash(d: dict):
  """Converts a dictionary of distinctive properties into a hash using md5"""
  arr = np.frombuffer(md5(json.dumps(d).encode()).digest(), dtype=np.uint8)[-8:]
  idx = np.arange(arr.shape[0], dtype=np.int64)[::-1]
  return np.asarray(np.sum(arr * 255**idx), dtype=np.int_).item()

In [ ]:
#| export
class cCrossSection(BaseModel):
  cs: CrossSection = Field(
    description="the contained CrossSection"
  )

  def __hash__(self):
    m = self.cs.cell.mx
    return dict_to_hash(dict(r=self.cs.mesh.bend_radius, m=hash(m.tostring())))

  def __eq__(self, other):
    return hash(self) == hash(other)

class cMode(BaseModel):
  mode: Mode = Field(
    description="the contained Mode"
  )
  
  def getMode(self) -> Mode:
    return self.mode

  def __hash__(self):
    return dict_to_hash(dict(neff=str(self.mode.neff), r=str(self.mode.cs.mesh.bend_radius)))
  
  def __eq__(self, other):
    return hash(self) == hash(other)

In [ ]:
#| export
@lru_cache(maxsize=None)
def inner(ccs, num_modes):
  return mw.compute_modes(ccs.cs, num_modes)

def cachedComputeModes(cs, num_modes):
  modes = inner(cCrossSection(cs=cs), num_modes)
  return [mode.copy(update={'cs': cs}) for mode in modes]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()